# Q-Learning für Verzerrer-Parameteroptimierung

In diesem Jupyter-Notebook verwenden wir Q-Learning, um die optimalen Parameter für einen Verzerrer zu finden. Der Code ist in verschiedene Sektionen unterteilt:

**1. Einrichtung und Initialisierung**
- **Bibliotheken**: Import von notwendigen Bibliotheken wie `numpy` und `scipy`.

**2. DistortionParameters-Klasse**
- **Zweck**: Repräsentiert die Parameter des Verzerrers: `Gain`, `Tone` und `Level`.
- **Methoden**: Zum Setzen und Abrufen dieser Parameter.

**3. Environment-Klasse: Verzerrer und Signal**
- **Signal**: Generiert ein Sinussignal und verarbeitet es mit dem Verzerrer.
- **Verarbeitung**: Methoden wie `_apply_gain`, `_apply_tone` und `_apply_level`.

**4. Q-Learning-Agent-Klasse**
- **Algorithmus**: Implementiert den Q-Learning-Algorithmus.
- **Q-Tabelle**: Wird zum Lernen der besten Aktionen (Parameter) verwendet.
- **Methoden**: Zum Wählen von Aktionen und Aktualisieren der Q-Tabelle.

**5. Training**
- **Prozess**: Trainiert den Q-Learning-Agenten über mehrere Episoden.
- **Belohnung**: In jeder Episode wird das Signal verarbeitet und eine Belohnung basierend auf dem Unterschied zum Ziel-Ausgangssignal berechnet.

**6. Ergebnisse**
- **Ausgabe**: Zeigt die gelernten optimalen Parameter nach dem Training.


### **Sektion 1: Einrichtung und Initialisierung**

In [3]:
import numpy as np
from scipy.signal import butter, lfilter
import random
import matplotlib.pyplot as plt
import scipy
from IPython.display import Audio


### **Sektion 2: DistortionParameters-Klasse**


In [4]:

class DistortionParameters:
    def __init__(self, gain=0.5, tone=0.5, level=0.5):
        self.gain = gain
        self.tone = tone
        self.level = level
    
    def set_parameters(self, gain, tone, level):
        self.gain = gain
        self.tone = tone
        self.level = level
        
    def get_parameters(self):
        return self.gain, self.tone, self.level

### **Sektion 3: Environment-Klasse: Verzerrer und Signal**

In [5]:
class DistortionEnvironment:
    def __init__(self, params):
        self.params = params
        self.signal = self._generate_signal()
        self.target_signal = self.process(self.signal)
    
    def _generate_signal(self):
        t = np.linspace(0, 1, 44100)
        return np.sin(2 * np.pi * 440 * t)
    
    def process(self, signal):
        return self._apply_level(self._apply_tone(self._apply_gain(signal, self.params.gain), self.params.tone), self.params.level)
    
    def _apply_gain(self, signal, gain):
        return np.clip(signal * gain, -1, 1)
    
    def _apply_tone(self, signal, tone):
        nyq = 0.5 * 44100
        low = 300
        high = 6000
        cutoff = low + (high - low) * tone
        b, a = butter(1, cutoff / nyq, btype='low')
        return lfilter(b, a, signal)
    
    def _apply_level(self, signal, level):
        return signal * level

### **Sektion 3: Environment-Klasse: Verzerrer und Signal**

In [6]:
class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.2):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((11, 11, 11))
    
    def choose_action(self):
        if np.random.uniform(0, 1) < self.epsilon:
            return np.random.rand(3)  # Random Action
        else:
            indices = np.unravel_index(np.argmax(self.q_table, axis=None), self.q_table.shape)  # Greedy Action
            return indices[0] / 10, indices[1] / 10, indices[2] / 10
    
    def update(self, state, reward):
        future_value = np.max(self.q_table)
        index = tuple((np.array(state) * 10).astype(int))
        self.q_table[index] = (
            self.q_table[index] + 
            self.alpha * (reward + self.gamma * future_value - self.q_table[index])
        )


### **Sektion 5: Training**

In [7]:
params = DistortionParameters(np.random.rand(), np.random.rand(), np.random.rand())
env = DistortionEnvironment(params)
agent = QLearningAgent()

def reward_function(output_signal, target_signal):
    return -np.mean((output_signal - target_signal) ** 2)

for episode in range(50000):
    action = agent.choose_action()
    params.set_parameters(*action)
    output_signal = env.process(env.signal)
    reward = reward_function(output_signal, env.target_signal)
    agent.update(action, reward)

### **Sektion 6: Ergebnisse**

In [8]:
optimal_indices = np.unravel_index(np.argmax(agent.q_table, axis=None), agent.q_table.shape)
optimal_params = DistortionParameters(optimal_indices[0] / 10, optimal_indices[1] / 10, optimal_indices[2] / 10)

print("Echte Parameter - Gain:", params.gain, "Tone:", params.tone, "Level:", params.level)
print("Gelernte Parameter - Gain:", optimal_params.gain, "Tone:", optimal_params.tone, "Level:", optimal_params.level)


Echte Parameter - Gain: 0.5 Tone: 0.1 Level: 1.0
Gelernte Parameter - Gain: 0.5 Tone: 0.1 Level: 1.0
